<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Troubleshooting Retrieval-Augmented Generation</h1>

Imagine you're an engineer at Arize AI and you've built and deployed a documentation question-answering service that uses retrieval-augmented generation to answer questions from your documentation. Users send questions about Arize's core product via a chat interface, and your service retrieves documents from your documentation in order to generate a response to the user. As the engineer in charge of evaluating and maintaining this system, you want to evaluate the quality of the responses from your service.

Phoenix helps you:
- identify gaps in your documentation
- detect queries for which the LLM gave bad responses
- detect failures to retrieve relevant context

In this tutorial, you will:

- Download an pre-indexed knowledge base of the Arize documentation
- Visualize user queries and knowledge base documents to identify areas of user interest not answered by your documentation
- Find clusters of responses with negative user feedback
- Identify failed retrievals using cosine similarity, Euclidean distance, and LLM-assisted ranking metrics

## Install Dependencies and Import Libraries

Install Phoenix.

In [ ]:
!pip install -q arize-phoenix

Import libraries.

In [ ]:
import pandas as pd
import phoenix as px

pd.set_option("display.max_colwidth", 1000)

## Download Your Knowledge Base and Query Datasets

Load your knowledge base.

In [ ]:
database_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/anthropic-hackathon/database_data.parquet"
)
database_df.head()

The columns of your dataframe are:

- **document_id:** the ID of the chunked document
- **text:** the chunked text in your knowledge base
- **text_vector:** the embedding vector for the text

Download a dataframe containing query data.

In [ ]:
query_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/context-retrieval/anthropic-hackathon/query_data.parquet"
)
query_df.head()

The columns of the dataframe are:

- **:id.id:**: the query ID
- **:timestamp.iso_8601:**: the time at which the query was made
- **:feature.text:prompt**: the query text
- **:feature.[float].embedding:prompt**: the embedding representation of the query
- **:prediction.text:response**: the final response presented to the user
- **:feature.[str].retrieved_document_ids:prompt**: the list of IDs of the retrieved documents
- **:feature.[float].retrieved_document_scores:prompt**: the lists of cosine similarities between the query and retrieved documents
- **:tag.float:user_feedback:** approval or rejection from the user (-1 means thumbs down, +1 means thumbs up)
- **:tag.str:relevance_0:** a binary classification (relevant vs. irrelevant) by an LLM predicting whether the first retrieved document is relevant to the query
- **:tag.str:relevance_1:** a binary classification (relevant vs. irrelevant) by an LLM predicting whether the second retrieved document is relevant to the query

The column names are in OpenInference format and describe the category, data type and intent of each column.

## Launch Phoenix

Define your knowledge base dataset with a schema that specifies the meaning of each column (features, predictions, actuals, tags, embeddings, etc.). See the [docs](https://docs.arize.com/phoenix/) for guides on how to define your own schema and API reference on `phoenix.Schema` and `phoenix.EmbeddingColumnNames`.

In [ ]:
database_schema = px.Schema(
    prediction_id_column_name="document_id",
    prompt_column_names=px.EmbeddingColumnNames(
        vector_column_name="text_vector",
        raw_data_column_name="text",
    ),
)
database_ds = px.Dataset(
    dataframe=database_df,
    schema=database_schema,
    name="database",
)

Define your query dataset. Because the query dataframe is in OpenInference format, Phoenix is able to infer the meaning of each column without a user-defined schema by using the `phoenix.Dataset.from_open_inference` class method.

In [ ]:
query_ds = px.Dataset.from_open_inference(query_df)

Launch Phoenix. Follow the instructions in the cell output to open the Phoenix UI.

In [ ]:
session = px.launch_app(primary=query_ds, corpus=database_ds)